In [ ]:
import myFunctions as mf
import pandas as pd
import numpy as np
import json

In [ ]:
myDataA = pd.read_csv("https://raw.githubusercontent.com/sunchang0124/PPDML/master/Privacy-preserving%20bayesians/preprocessed_dataFile_A.csv")
myDataA = myDataA.drop('Unnamed: 0', axis=1)

In [ ]:
# We assume target class is at B site and always the last column in B data
myDataB = pd.read_csv("https://raw.githubusercontent.com/sunchang0124/PPDML/master/Privacy-preserving%20bayesians/preprocessed_dataFile_B.csv")
colB = myDataB.columns
Y = myDataB['diag_3']
myDataB = myDataB[colB[0:6]].drop(['Unnamed: 0'], axis=1) # we only use first 5 columns here because the values    


In [ ]:
Divide_set = 10
C_seed = 2
C_min = 0
C_max = 5

In [ ]:
resultStageOne = mf.start_at_A(myDataA, Divide_set, C_seed, C_min, C_max)

In [ ]:
resultStageTwo = mf.start_at_B(myDataB, C_seed, C_min, C_max, resultStageOne["sumNoiseBytes"], Divide_set)

In [ ]:
resultStageThree = mf.communication_at_A(myDataA, resultStageOne['randomBytes'], resultStageTwo['sumNoisesAB'], \
                                         resultStageTwo['sumNoisesB'], Divide_set)

In [ ]:
myResult = mf.Final_at_B(myDataB, resultStageThree["randomsSumSet"], resultStageThree["sumNoisesBARand"], \
                         resultStageThree["XaTXa"], resultStageTwo['randomBytes'], Divide_set)


In [ ]:
##### checking with centralized datasets by using scikit learn #####

X_centralized = np.concatenate((myDataA, myDataB.drop('diag_3', axis=1)), axis=1)
from sklearn import linear_model
regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
regr.fit(X_centralized, Y)

# The coefficients
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

In [ ]:
for i in range(0, len(myResult["coefficients"])):
    print(str(np.matrix(regr.coef_[i+1])) == str(myResult["coefficients"][i]))